# Symbolize - Type Theory Logic - New Style

In [1]:
import sys
sys.path.insert(1, "../..")
from symbolize.logic.argument import Argument
from symbolize.logic.typetheory.proposition import *
from symbolize.logic.typetheory.proof import *
from symbolize.logic.typetheory.variables import A, B, C, Verum, Falsum

## Conjunction ##
### Introduction ###

In [2]:
p = A.get_proof('p')
q = B.get_proof('q')
r = ProofExpressionCombination(p,q)
Argument([p,q], r)

### Elimination ###

In [3]:
Argument([r], fst(r))

In [4]:
A_and_B = and_(A, B)
r = A_and_B.get_proof('r')
Argument([r], fst(r))

In [5]:
snd(r)

snd(r) : B

#### Example - From $A \lor B$ deduce $B \lor A$ ####

In [6]:
Argument([
    Argument([r], fst(r)),
    Argument([r], snd(r))
],
    ProofExpressionCombination(
        fst(r),
        snd(r)
    )
)

## Implication ##
### Introduction ###

In [7]:
x = A.get_proof('x')
e = B.get_proof('e')
Argument([e], e.abstract(x), discharges=[x], label=r"\Rightarrow{I}")

### Elimination ###

In [8]:
A_implies_B = implies(A, B)
q = A_implies_B.get_proof('q')
a = A.get_proof('a')
q(a)

q(a) : B

## Disjunction ##
### Introduction ###

In [9]:
q = A.get_proof('q')
inl(q, B)

inl(q, B) : ∨(A, B)

In [10]:
r = B.get_proof('r')
inr(r, A)

inr(r, A) : ∨(A, B)

In [11]:
B.arity

∅

#### Example - Identity - [ST] p83 ####

In [12]:
Argument([x], x.abstract(x), discharges=[x], label=r"\Rightarrow{I}")

#### Example - From $A \Rightarrow B$ and $B \Rightarrow C$ deduce $A \Rightarrow C$ - [ST] p83-84 ####

In [13]:
A_implies_B = implies(A, B)
B_implies_C = implies(B, C)
a = A_implies_B.get_proof('a')
b = B_implies_C.get_proof('b')
x = A.get_proof('x')

In [14]:
Argument([x, a], a(x))

In [15]:
arg1 = Argument([x, a], a(x))
arg2 = Argument([arg1, b], b(arg1.conclusion))
arg3 = Argument([arg2, x], arg2.conclusion.abstract(x))
arg4 = Argument([arg2, x], arg3.conclusion.abstract(b))
arg5 = Argument([arg2, x], arg4.conclusion.abstract(a))
arg5

In [16]:
arg5.conclusion.proposition_type.substitute(C, Falsum)

⟹(⟹(A, B), ⟹(⟹(B, ⟘), ⟹(A, ⟘)))

In [17]:
foo = arg5.conclusion.proposition_type.substitute(C, Falsum).copy()
def my_sub(wr):
    if type(wr.expr) is PropositionBinaryInfixExpression and wr.expr.base == implies and wr.expr.children[1] == Falsum:
        wr.obj[wr.index] = not_(wr.expr.children[0])
foo.walk(my_sub)
foo

⟹(⟹(A, B), ⟹(⟹(B, ⟘), ⟹(A, ⟘)))

#### Example - $((A \lor B) \Rightarrow C) \iff ((A \Rightarrow C) \land (B \Rightarrow C))$ - [ST] p86-87 ####
##### $((A \lor B) \Rightarrow C) \Rightarrow ((A \Rightarrow C) \land (B \Rightarrow C))$ #####

In [18]:
A_or_B_implies_C = implies(or_(A, B), C)
y = A_or_B_implies_C.get_proof('y')
x = A.get_proof('x')
w = B.get_proof('w')

In [19]:
arg1 = Argument([x], inl(x, B))
arg2 = Argument([arg1, y], y(arg1.conclusion))
arg3 = Argument([arg2], arg2.conclusion.abstract(x))

arg4 = Argument([w], inr(w, A))
arg5 = Argument([arg4, y], y(arg4.conclusion))
arg6 = Argument([arg5], arg5.conclusion.abstract(w))

arg7 = Argument([arg3, arg6], ProofExpressionCombination(arg3.conclusion, arg6.conclusion))
arg7

In [20]:
arg7.conclusion.abstract(y)

(y)(x)y(inl(x, B)), (w)y(inr(w, A)) : ⟹(⟹(∨(A, B), C), ∧(⟹(A, C), ⟹(B, C)))

##### $((A \lor B) \Rightarrow C) \Leftarrow ((A \Rightarrow C) \land (B \Rightarrow C))$ #####

In [21]:
A_or_B = or_(A, B)
A_implies_C_and_B_implies_C = and_(implies(A, C),implies(B, C))
z = A_or_B.get_proof('z')
p = A_implies_C_and_B_implies_C.get_proof('p')

In [22]:
p

p : ∧(⟹(A, C), ⟹(B, C))

In [23]:
Argument([z, p.select(0).alias("fst~p"), p.select(1).alias("snd~p")],
  cases(z, p.select(0).alias("fst~p"), p.select(1).alias("snd~p"))       
)

In [24]:
cases(z, p.select(0).alias("fst~p"), p.select(1).alias("snd~p")).abstract(z).abstract(p)

(p)(z)cases(z, fst~p, snd~p) : ⟹(∧(⟹(A, C), ⟹(B, C)), ⟹(∨(A, B), C))

## Universal Quantifier ##
### Introduction ###

In [25]:
x = A.get_proof('x')
P = PropositionSymbol('P', assumption_contains_free=[x])
p = P.get_proof('p')
Argument([p], p.abstract(x), discharges=[x], label=r"\forall{I}")

### Elimination ###

In [26]:
a = A.get_proof('a')
forall_x_P = forall(x, P)
f = forall_x_P.get_proof('f')

Argument([a,f], f.apply(a), label=r"\forall{E}")

## Existential Quantifier ##
### Introduction ###

In [32]:
a = A.get_proof('a')
x = Symbol('x')
P = PropositionSymbol('P', assumption_contains_free=[x])
p = P.get_proof('p')
Argument([a,p], ProofExpressionCombination(a,p, exists_expression=x), label=r"\exists{I}")

### Elimination ###

In [33]:
x = A.get_proof('x')
P = PropositionSymbol('P', assumption_contains_free=[x])

exists_x_P = exists(x, P)
p = exists_x_P.get_proof('p',  exists_expression=x)


Argument([p], Fst(p), label=r"\exists{E_1}")

In [34]:
Argument([p], Snd(p), label=r"\exists{E_2}")

### [ST] p92 ###

In [37]:
x = A.get_proof('x')
B = PropositionSymbol('B', assumption_contains_free=[x])
C = PropositionSymbol('C', assumption_contains_free=[x])
B_implies_C = implies(B, C)
forall_x_B_implies_C = forall(x, B_implies_C)
r = forall_x_B_implies_C.get_proof('r')
forall_x_B = forall(x, B)
p = forall_x_B.get_proof('p')

In [ ]:
Argument([r, x], r.apply(x), label=r"\forall{E}")

In [ ]:
Argument([x, p], p.apply(x), label=r"\forall{E}")